In [ ]:
from hyperopt import * #probably do not need to import everything
import pickle
import time
import numpy as np

import pandas as pd
import os
import csv
import statistics       
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.model_selection import GroupShuffleSplit


from sklearn.svm import LinearSVC

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import LeaveOneGroupOut
from sklearn import model_selection, naive_bayes, svm


from sklearn.metrics import make_scorer

from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split


from sklearn.metrics import cohen_kappa_score, make_scorer
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

from sklearn.model_selection import GroupKFold



from sklearn.model_selection import cross_val_predict
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score



import re
import nltk
import contractions
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer


: 

In [ ]:
pd.set_option('display.max_columns', None)


: 

In [ ]:
!hostname



In [ ]:
!ls

In [ ]:


hunting_set_file ='/s/babbage/b/nobackup/nblancha/public-datasets/ilideep/natesTask/ThotStuff/Data/FinalThots.xlsx'


df_hunting = pd.read_excel(hunting_set_file)
df_hunting.dtypes

In [ ]:
import pandas as pd


df_hunting = df_hunting[~df_hunting['Type'].isin(['DV_recall', 'IAM_recall'])]

In [ ]:
df_hunting



In [ ]:

def clean(text):
# Removes all special characters and numericals leaving the alphabets
    text = contractions.fix(text)
    #text = re.sub('[^A-Za-z]+', ' ', text)
    word_tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    #start_words = ['not', 'no', 'nor', 'against']
    filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
    return ' '.join(filtered_sentence)


wnl = WordNetLemmatizer()
##The give words to take out
giveWords = ['deja-vu',
             'deja',
             'déjà',
             'vu',
             'memory' ,
             'remembered' ,
             'thoughts',
             'involuntary' ,
             'pop',
             'popped',
             'pops',
             'popping',
             'recall',
             'recalled',
             'remember',
             'remembering',
             'memories',
             'involuntarily',
             'sudden',
             'thought',
             'unexpected',
             'felt',
             'experience',
             'unexpecting',
             'suddenly',
             'think',
             'feeling',
             'feel']
#giveWords = []
listOfSentences = df_hunting['Sentence'] 
correctSentences = [] 
for sentence in listOfSentences: 
  
  filtered_words = [word for word in sentence.split() if word.lower()  not in giveWords]
  final_string = ' '.join(filtered_words)
  correctSentences.append(final_string)
  #print(final_string)
df_hunting['Sentence'] = correctSentences
df_hunting['Sentence'] = df_hunting['Sentence'].apply(clean)

In [ ]:
df_hunting


In [ ]:
df_hunting

In [ ]:
#Features from the dataset
#Row start : Row end , Column Start , Column end 
from sklearn.feature_extraction.text import TfidfVectorizer
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(df_hunting['Sentence'])
X_Tfidf = Tfidf_vect.transform(df_hunting['Sentence'])
#df_hunting['embedding'] = X_Tfidf.toarray()
df_hunting = pd.concat([df_hunting,pd.DataFrame(X_Tfidf.todense(), index=df_hunting.index)], axis=1)
#pd.DataFrame(X_Tfidf.todense())
#df_hunting

In [ ]:
X = df_hunting.iloc[:, 9:]
#X = df_hunting.iloc[  : , 13: ]
print(X.columns)
X = X.to_numpy()
X = X_Tfidf.toarray() 

In [ ]:
df_hunting

In [ ]:
X.shape

In [ ]:
X_Tfidf.shape


In [ ]:
#Labels from the dataset
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(df_hunting['Type'])
df_hunting['Type'] = y_encoded

y = pd.DataFrame(df_hunting['Type'], columns=['Type'])
y = y.to_numpy()
y=y.reshape(-1)
#y_encoded
y

In [ ]:
df_hunting['Type'] 

In [ ]:
df_hunting

In [ ]:
#some instances need to be grouped
#df_hunting = df_hunting.sort_values("id")
groups = pd.DataFrame(df_hunting['Subject_id'], columns=['Subject_id'])
#print(groups)
groups = groups.to_numpy()
groups = groups.reshape(-1)

#this is cross-validation iterator
#cv = GroupShuffleSplit(n_splits=59, test_size=1, random_state=42)



In [ ]:
print(groups)

In [ ]:
participants = np.unique(groups)
#print(participants)
len(participants)

In [ ]:
print (X.shape, y.shape)



In [ ]:
!hostname

output_csv = "Language-PredictAge-7-13-2023.csv"
results_path = r"/s/babbage/b/nobackup/nblancha/public-datasets/ilideep/natesTask/ThotStuff/Important"
modelcsv =os.path.join(results_path, output_csv)
if not os.path.exists(modelcsv):
        os.mkdir(modelcsv)

In [ ]:

# group_kfold = GroupKFold(n_splits=2)
logo = LeaveOneGroupOut()
cv = logo#.split(X, y, groups=groups) #group_kfold #

y_pred = cross_val_predict(AdaBoostClassifier(algorithm = 'SAMME', learning_rate =1.8142857142857143, n_estimators = 74), X, y, groups=groups, n_jobs=20,cv=cv, method='predict')
#{'algorithm': 'SAMME', 'learning_rate': 1.8142857142857143, 'n_estimators': 74}
#'algorithm': 'SAMME', 'learning_rate': 4.1, 'n_estimators': 74
#'algorithm': 'SAMME.R', 'learning_rate': 3.610204081632653, 'n_estimators': 74
#print(y)
f1_hunt = f1_score(y, y_pred, average=None)[0]
f1_non_hunt = f1_score(y, y_pred, average=None)[1]
acc_overall = accuracy_score(y, y_pred)
acc_balanced = balanced_accuracy_score(y, y_pred)
kappa = cohen_kappa_score(y, y_pred)
roc = roc_auc_score(y, y_pred)
cm = confusion_matrix(y, y_pred)
cm_0_0 = cm[0][0]
cm_0_1 = cm[0][1]
cm_1_0 = cm[1][0]
cm_1_1 = cm[1][1]
scoring = {
    'f1': make_scorer(f1_score, average='weighted'),  # You can adjust 'average' as needed
    'kappa': make_scorer(cohen_kappa_score),
    'accuracy': make_scorer(accuracy_score)
    }
scores = cross_validate(AdaBoostClassifier(algorithm = 'SAMME', learning_rate = 1.8142857142857143, n_estimators = 74), X, y,groups=groups, n_jobs=20,cv=cv,scoring=scoring)
print("Mean F1 Score:", scores['test_f1'].mean())
print("Mean Kappa Score:", scores['test_kappa'].mean())
print("Mean Accuracy Score:", scores['test_accuracy'].mean())
averageF1 = scores['test_f1'].mean()
avgerageKappa = scores['test_kappa'].mean()
avgerageAccuracy = scores['test_accuracy'].mean()
stdF1 = scores['test_f1'].std()
stdKappa = scores['test_kappa'].std()
stdAccuracy = scores['test_accuracy'].std()

In [ ]:
cm

In [ ]:
accuracy_score(y,y_pred)

In [ ]:
f1_score(y, y_pred, average='weighted')

In [ ]:
clf = AdaBoostClassifier(algorithm = 'SAMME.R', learning_rate = 3.610204081632653, n_estimators = 74)
clf.fit(X, y)

# get feature importance scores
feature_importances = clf.feature_importances_
vectorizer = TfidfVectorizer()
# map feature importance scores to feature names
feature_names = np.array(vectorizer.get_feature_names())
sorted_idx = feature_importances.argsort()[::-1]
feature_names[sorted_idx]

# print top 10 important features
print("Top 10 important features:")
for feature in feature_names[sorted_idx][:10]:
    print("- {}".format(feature))


In [ ]:
logo = LeaveOneGroupOut()
logo.get_n_splits(X, y, groups)
cv = logo


In [ ]:
import os
os.environ['PYSPARK_PYTHON'] = '/s/babbage/b/nobackup/nblancha/merry/conda/envs/hyperTesting/bin/python'
os.environ['PYSPARK_DRIVER'] = '/s/babbage/b/nobackup/nblancha/merry/conda/envs/hyperTesting/bin/python'

In [ ]:
output_csv = "8-26-2023_predictType"
results_path = "./"
modelcsv =os.path.join(results_path, output_csv)
if not os.path.exists(modelcsv):
        os.mkdir(modelcsv)
else:
        print("WARNING: DIRECTORY ALREADY EXISTS")

In [ ]:
from pyspark.sql import SparkSession
avg = [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0]
results = []
group_kfold = GroupKFold(n_splits=len(participants))
# group_kfold = GroupKFold(n_splits=2)
logo = LeaveOneGroupOut()
cv = logo#.split(X, y, groups=groups) #group_kfold #

#avg = [1.0,2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0]
avg = [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0]
print (X.shape, y.shape)

best = 0

final_score = { 
               'model_name':[], 
               'hyper_paramaters':[],
               'f1_0':[],
               'acc_overall': [],
               'acc_balanced':[], 
               'kappa':[],
               'roc':[],
               'roc_w':[], 
               'cm': [],
               'roc_avg':[]}

def hyperopt_train_test(params, groups, cv):
    global avg
    global all_results
    global final_score
    listOfScores = []
    model_name = params['type']
    predictioncsv = model_name
    del params['type']
    for item in params.values():
        predictioncsv += '_' + str(item)
    predictioncsv += '.csv'
    
    if model_name == 'naive_bayes':
        clf = GaussianNB(**params)
    elif model_name == 'AdaBoost':
        clf = AdaBoostClassifier(**params)
    elif model_name == 'LinSVC':
        clf = LinearSVC(**params)      
    elif model_name == 'decision_tree':
        clf = DecisionTreeClassifier(**params, random_state=1) #(**params, n_jobs=6)
    elif model_name == 'randomforest':
        clf = RandomForestClassifier(**params, random_state=1) #(**params, n_jobs=6)
    elif model_name == 'knn':
        clf = KNeighborsClassifier(**params)
    elif model_name == 'logistic_regression':
        clf = LogisticRegression(**params, random_state=1)
    elif model_name == 'SVC':
        clf = SVC(**params, probability=True)
    elif model_name == 'SVM':
        clf = svm.SVC(**params, gamma='auto',probability=True)
    else:
        return 0
    #scores =  cross_validate(clf, X, y,scoring=scoring, groups=groups, cv=cv, n_jobs=20, verbose=200)
    y_pred = cross_val_predict(clf, X, y, groups=groups, n_jobs=20,cv=cv, verbose=0, method='predict')
    #y_pred = cross_val_predict(clf, X, y, n_jobs=20,cv=cv, verbose=0, method='predict')
    y_score = cross_val_predict(clf, X, y, groups=groups, n_jobs=20,cv=cv, verbose=0, method='predict_proba') # Dont this this is ever used
    scoring = {
    'f1': make_scorer(f1_score, average='weighted'),  # You can adjust 'average' as needed
    'kappa': make_scorer(cohen_kappa_score),
    'accuracy': make_scorer(accuracy_score)
    }
    scores = cross_validate(clf, X, y,groups=groups, n_jobs=20,cv=cv,scoring=scoring)
    print("Mean F1 Score:", scores['test_f1'].mean())
    print("Mean Kappa Score:", scores['test_kappa'].mean())
    print("Mean Accuracy Score:", scores['test_accuracy'].mean())
    averageF1 = scores['test_f1'].mean()
    avgerageKappa = scores['test_kappa'].mean()
    avgerageAccuracy = scores['test_accuracy'].mean()
    stdF1 = scores['test_f1'].std()
    stdKappa = scores['test_kappa'].std()
    stdAccuracy = scores['test_accuracy'].std()
    listOfScores.append(scores)
    f1_0 = f1_score(y, y_pred, labels=[0,1,2], average='weighted')
    #print('This is y_pred',y_pred)
    acc_overall = accuracy_score(y, y_pred)
    
    acc_balanced = balanced_accuracy_score(y, y_pred)
    kappa = cohen_kappa_score(y, y_pred)
    cm = confusion_matrix(y, y_pred)
    ###
    roc_auc_ovr = {}
    weighted_roc = {}
    roc_avg = 0
    for class_id in range(2):
        print(y_score.shape)
        prob_tmp = y_score[:, class_id]
        true_max_tmp = [1 if y_tmp == class_id else 0 for y_tmp in y]
        tpr_list = [0]
        fpr_list = [0]
        try:
            roc_auc_ovr[class_id] = roc_auc_score(true_max_tmp, prob_tmp)
            weighted_roc[class_id] = roc_auc_score(true_max_tmp, prob_tmp, average='weighted')
            if(class_id > 0): roc_avg += weighted_roc[class_id]
        except:
            print(f"Issue with class {class_id}")
            roc_auc_ovr[class_id] = None
    roc = roc_avg/4

    avg = [model_name, params, f1_0, acc_overall, acc_balanced, kappa, roc_auc_ovr, weighted_roc, cm, roc_avg, y_pred, predictioncsv,y,averageF1,avgerageAccuracy,
           avgerageKappa,stdF1,stdAccuracy,stdKappa]
   
    return avg[2], avg
    
#define your search space
space = hp.choice('classifier_type', [
    {
        'type': 'AdaBoost',
        'n_estimators': hp.choice('AdaBoost_n_estimators', list(range(30, 100, 11))),
        'learning_rate': hp.choice('AdaBoost_learning_rate', list(np.linspace(0.1, 4.1, 50))),
        'algorithm': hp.choice('AdaBoost_algorithm', ['SAMME', 'SAMME.R'])
    },
    
    {
        'type': 'naive_bayes',
       
    },
    {
        'type': 'logistic_regression',
        'penalty' : hp.choice('logistic_regression_penalty', ['l2', 'none']),
        'solver' : hp.choice('logistic_regression_solver', ['lbfgs', 'newton-cg', 'sag', 'saga'])
    },
    {
        'type': 'SVC',
        'C': hp.choice('C', [0.1, 1, 10, 100, 1000]),
        'gamma': hp.choice('gamma', [1, 0.1, 0.01, 0.001, 0.0001]),
        'kernel': hp.choice('kernal', ['rbf', 'sigmoid']),

    },
#     {
# 
#    
#     {
#         'type': 'decision_tree',
#         'criterion_dt': hp.choice('criterion', ["gini", "entropy", "log_loss"]),
#         'splitter': hp.choice('splitter', ["best", "random"]),
#         'max_features': hp.choice('max_features', ["auto", "sqrt", "log2"]),
#
#     },
  
    # {
    #    'type': 'SVM',
    #    'C': hp.loguniform('C', -5, 2),  # Penalty parameter C
    #    'kernel': hp.choice('kernel', ['linear', 'poly', 'rbf', 'sigmoid']),  # Kernel type
    #    'degree': hp.choice('degree', np.arange(1, 13+1, dtype=int)),  # Degree of the polynomial kernel
    #   # 'gamma': hp.loguniform('gamma', -5, 2)  # Kernel coefficient for 'rbf', 'poly', and 'sigmoid'
       
    # },
   {
       'type': 'randomforest',
       'n_estimators': hp.choice('n_estimators', list(range(20, 170, 16))),
       'criterion': hp.choice('criterion', ["gini", "entropy"]),
       'max_features': hp.choice('max_features', [None, "sqrt", "log2"]),
       
   },
    {
        'type': 'knn',
        'n_neighbors': hp.choice('knn_n_neighbors', list(range(1, 300, 10)))
    }
    
])
count = 1
all_results =[]

trials = Trials()
def f(params):
    global best, count, avg, all_results, final_score
    count += 1
    guide_metric, scores = hyperopt_train_test(params.copy(), groups, cv)
    
    results.append(scores)
    if guide_metric > best:
        best = guide_metric
    else: 
        print (scores)
    print(avg[13])
        
    #print ('iters:', count, ', kappa:', guide_metric, 'using', params)
    return {'loss': -guide_metric, 'status': STATUS_OK, 'eval_time': time.time(), 
            'other_stuff': {'model':avg[0],'params':avg[1],'f1_0':avg[2] ,'acc_overall': avg[3],
                           'acc_balanced': avg[4],'kappa':avg[5] , 'roc': avg[6],'roc_w': avg[7], 'cm': avg[8], 
                           'roc_avg': avg[9], 'y_pred': avg[10], 'predcsv' : avg[11],'y_actual':avg[12],'averageF1':avg[13],'avgerageAccuracy':avg[14], 
                            'avgerageKappa':avg[15], 'stdF1':avg[16], 'stdAccuracy':avg[17],'stdKappa':avg[18] }}
# avg = [model_name, params, f1_0, acc_overall, acc_balanced, kappa, roc_auc_ovr, weighted_roc, cm, roc_avg, y_pred, predictioncsv,y,averageF1,avgerageAccuracy,
           #avgerageKappa,stdF1,stdAccuracy,stdKappa]
#spark = SparkSession.builder.master("spark://luffy:30122").getOrCreate()...,
#spark_trials = SparkTrials(parallelism=8, spark_session=spark) #DO NOT USE
spark_trials = SparkTrials(parallelism=128)

best = fmin(
            f, 
            space, 
            trials=spark_trials,    
            algo=tpe.suggest, 
            max_evals= 2000,)
print ('best:' )
print(best)
#performance_csv = os.path.join("models", "performance.csv")
#pd.DataFrame(final_score).to_csv(performance_csv, index=False)

# -> {'a': 1, 'c2': 0.01420615366247227}
#print(space_eval(space, best))
# -> ('case 2', 0.01420615366247227)
#df = pd.DataFrame(results, columns=["model_name", "params", "f1_0", "acc_overall", "acc_balanced", "kappa", "roc_auc_ovr", "weighted_roc", "cm", "roc_avg", 'y_pred', 'predcsv'])
#df = df.sort_values(by="f1_0", ascending=False)
#df.to_csv(modelcsv)
#df.to_csv(modelcsv + ".csv")


In [ ]:
avg

In [ ]:
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import GroupKFold

# Assuming you have your data organized by groups
groups = groups # List of group IDs for each sample

# Define your classifier 'clf'
clf = clf = RandomForestClassifier(n_estimators=64, criterion='gini',max_depth=56) # Instantiate your classifier here

# Perform cross-validated predictions with GroupKFold
cv = GroupKFold(n_splits=314)
fold_predictions_list = []

for fold, (train_indices, test_indices) in enumerate(cv.split(X, y, groups)):
    X_train, y_train = X[train_indices], y[train_indices]
    X_test = X[test_indices]
    
    clf.fit(X_train, y_train)
    fold_predictions = clf.predict(X_test)
    fold_predictions_list.append(fold_predictions)

In [ ]:
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession



# Configure Spark to connect to the existing cluster
conf = SparkConf().setMaster("spark://luffy:30122")

# Create SparkSession with the configured options
spark = SparkSession.builder.config(conf=conf).getOrCreate()

In [ ]:
spark.stop()

In [ ]:
len(spark_trials.results)

In [ ]:
spark_trials.trials

In [ ]:
best_results = []
for trial in spark_trials.trials:
    best_results.append(trial['result']['other_stuff'])
    

In [ ]:
listOfAllModels = []
for eachModel in best_results:
    df = pd.DataFrame([eachModel])
    print(df)
    listOfAllModels.append(df)
    

In [ ]:
concatenated_df = pd.concat(listOfAllModels, ignore_index=True)
#df.to_csv(csv_file_path, index=False)

In [ ]:
concatenated_df.shape

In [ ]:
concatenated_df.to_csv('3-13-24-PredictTypeUsingLanguage.csv')

In [ ]:
y_pred = concatenated_df['y_pred']
y_actual = concatenated_df['y_actual']

In [ ]:
np.array(y_actual)

In [ ]:
y_pred

In [ ]:
cohen_kappa_score(np.array(y_actual[0]),np.array(y_pred[0]))

In [ ]:
X